## Imports

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt
import hvplot.pandas
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve
import holoviews as hv
from fiona.crs import from_epsg
from geopy.geocoders import Nominatim
import leafmap
import warnings
warnings.filterwarnings('ignore')
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
#hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}





In [3]:
pd.set_option('display.max_colwidth', None) # To avoid truncated display of DataFrames 

## Reading Data

In [4]:
df = pd.read_excel("PositionsUnit-LongLat.xlsx", index_col = [0])
    

In [414]:
df #703 632507601 2023-04-06 09:21:40+02:00 AND 631235279	2023-03-31 06:54:45+02:00

,UnitPositionId,DateTimeOfPosition,IgnitionOn,long,lat
0,637458912,2023-05-01 06:29:18.987 +02:00,0,N52.154859,E5.376457
1,637458804,2023-05-01 06:28:58.437 +02:00,1,N52.154865,E5.376504
2,637458106,2023-05-01 06:26:58.420 +02:00,1,N52.158796,E5.379862
3,637457374,2023-05-01 06:24:58.953 +02:00,1,N52.152371,E5.388205
4,637456653,2023-05-01 06:22:58.207 +02:00,1,N52.157829,E5.404613
...,...,...,...,...,...
995,630553518,2023-03-28 10:22:19.030 +02:00,1,N52.178164,E5.468254
996,630553514,2023-03-28 10:22:18.600 +02:00,1,N52.178164,E5.468254
997,630553513,2023-03-28 10:22:18.600 +02:00,1,N52.176882,E5.463401
998,630553512,2023-03-28 10:22:18.600 +02:00,1,N52.178559,E5.459197


### Convert str to flaot

In [5]:
#Convert Longs and Lats to floats. 

def cnvrtCoord(val: str) -> float:
    sgn = +1
    if val[0] == 'S' or val[0] == 'W':
        sgn = -1
    return float(val[1:]) * sgn  

df['long'] = [cnvrtCoord(x) for x in df['long'].tolist()]
df['lat'] = [cnvrtCoord(x) for x in df['lat'].tolist()]  


In [41]:
# Convert the DataFrame to a GeoDataFrame
# geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
# gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Print the resulting GeoDataFrame
#print(gdf)


### Convert str to datetime


In [6]:
# Another cleaned DF. 

df['DateTimeOfPosition'] = pd.to_datetime(df['DateTimeOfPosition'])


In [525]:
#df[950:]

## Create Trajectory


In [7]:
#gdf.set_index("DateTimeOfPosition", inplace=True)
traj = mpd.Trajectory(df[["DateTimeOfPosition", "long", "lat"]], "Allsetra", x='lat', y='long', t='DateTimeOfPosition', crs=4326)


In [8]:
traj

Trajectory Allsetra (2023-03-28 06:32:06.247000 to 2023-05-01 06:29:18.987000) | Size: 986 | Length: 1150621.7m
Bounds: (4.115409, 51.339758, 5.674972, 52.33138)
LINESTRING (5.459454 52.178471, 5.468254 52.178164, 5.468254 52.178164, 5.468272 52.178172, 5.468269

In [9]:
traj.df

,geometry
DateTimeOfPosition,
2023-03-28 06:32:06.247,POINT (5.45945 52.17847)
2023-03-28 10:22:18.600,POINT (5.46825 52.17816)
2023-03-28 10:22:19.030,POINT (5.46825 52.17816)
2023-03-28 10:22:33.527,POINT (5.46827 52.17817)
2023-03-28 10:44:30.883,POINT (5.46827 52.17810)
...,...
2023-05-01 06:22:58.207,POINT (5.40461 52.15783)
2023-05-01 06:24:58.953,POINT (5.38821 52.15237)
2023-05-01 06:26:58.420,POINT (5.37986 52.15880)


## Trajectory Viz


In [423]:
traj.hvplot(line_width=8, tiles='EsriStreet') #EsriImagery

#.hvplot(line_width = 4).opts(projection='mercator')

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (triangle_angle)

## StopDetection


In [425]:
from datetime import timedelta

In [426]:
detector = mpd.TrajectoryStopDetector(traj)

In [427]:
stops = detector.get_stop_points(min_duration=timedelta(seconds=200), max_diameter=200)


In [428]:
stops

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-03-28 10:22:18.600000,POINT (5.46826 52.17816),2023-03-28 10:22:18.600,2023-03-28 10:44:30.883,Allsetra,1332.283
Allsetra_2023-03-28 10:48:56.750000,POINT (5.45512 52.17664),2023-03-28 10:48:56.750,2023-03-28 11:12:33.617,Allsetra,1416.867
Allsetra_2023-03-28 11:14:24.630000,POINT (5.45953 52.17846),2023-03-28 11:14:24.630,2023-03-28 15:06:24.687,Allsetra,13920.057
Allsetra_2023-03-28 15:08:25.110000,POINT (5.46073 52.17566),2023-03-28 15:08:25.110,2023-03-28 15:12:25.553,Allsetra,240.443
Allsetra_2023-03-28 15:14:25.437000,POINT (5.45952 52.17835),2023-03-28 15:14:25.437,2023-03-29 08:50:23.957,Allsetra,63358.520
...,...,...,...,...,...
Allsetra_2023-04-28 12:45:39.883000,POINT (4.86973 52.29563),2023-04-28 12:45:39.883,2023-04-28 12:57:02.930,Allsetra,683.047
Allsetra_2023-04-28 13:36:52.403000,POINT (5.45949 52.17846),2023-04-28 13:36:52.403,2023-04-28 15:20:04.613,Allsetra,6192.210
Allsetra_2023-04-28 15:26:04.340000,POINT (5.42272 52.16301),2023-04-28 15:26:04.340,2023-04-28 15:38:04.677,Allsetra,720.337


## Stop Viz


In [101]:
stops = stops.set_crs(4326)

In [429]:
import hvplot.pandas



In [430]:
stops.hvplot(geo=True, tiles=True, hover_cols="all", frame_height=300, frame_width=400) +\
traj.hvplot(geo=True, tiles=True,  frame_height=300, frame_width=400)

:Layout
   .Overlay.I  :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Points.I :Points   [Longitude,Latitude]   (stop_id,start_time,end_time,traj_id,duration_s)
   .Overlay.II :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Path.I   :Path   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]   (triangle_angle)

## Getting Address from Long-Lat Coordinates

In [431]:
geolocator = Nominatim(user_agent="Allsetra-Tracking")

def get_address_from_GeoPoint(point): 
    query = ",".join([str(point.y), str(point.x)])
    location = geolocator.reverse(query)
    #print(f"Point: {point}, Query: {query}, Addr: {location.address}")
        
    return location.address

In [432]:
traj.df["Address"] = traj.df.geometry.apply(lambda x: get_address_from_GeoPoint(x))

In [246]:
#traj.df["Address"] = get_address_from_GeoPoint(traj.df.geometry)

In [433]:
traj.df

,geometry,Address
DateTimeOfPosition,,
2023-03-28 06:32:06.247,POINT (5.45945 52.17847),"44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland"
2023-03-28 10:22:18.600,POINT (5.46825 52.17816),"Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland"
2023-03-28 10:22:19.030,POINT (5.46825 52.17816),"Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland"
2023-03-28 10:22:33.527,POINT (5.46827 52.17817),"Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland"
2023-03-28 10:44:30.883,POINT (5.46827 52.17810),"Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland"
...,...,...
2023-05-01 06:22:58.207,POINT (5.40461 52.15783),"102, Hogeweg, De Kruiskamp, Amersfoort, Utrecht, Nederland, 3814 CG, Nederland"
2023-05-01 06:24:58.953,POINT (5.38821 52.15237),"Stadsring, Stadskern, Amersfoort, Utrecht, Nederland, 3817 AX, Nederland"
2023-05-01 06:26:58.420,POINT (5.37986 52.15880),"De Nieuwe Poort, De Nieuwe Stad, Isselt, Amersfoort, Utrecht, Nederland, 3812 PA, Nederland"


## Stop-Trajectory Splitter 

In [13]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=200, min_duration=timedelta(seconds=200))


In [14]:
traj_col.hvplot(line_width = 10)

:Overlay
   .Tiles.I         :Tiles   [x,y]
   .Path.I          :Path   [Longitude,Latitude]
   .Points.I        :Points   [Longitude,Latitude]   (triangle_angle)
   .Path.II         :Path   [Longitude,Latitude]
   .Points.II       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle)
   .Path.III        :Path   [Longitude,Latitude]
   .Points.III      :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle)
   .Path.IV         :Path   [Longitude,Latitude]
   .Points.IV       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.V          :Path   [Longitude,Latitude]
   .Points.V        :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.VI         :Path   [Longitude,Latitude]
   .Points.VI       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.VII        :Path   [Longitude,Latitude]
   .Points.VII      :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.VIII       :Path   [Longitude,Latitude]
   .Points.VIII     :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.IX         :Path   [Longitude,Latitude]
   .Points.IX       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.X          :Path   [Longitude,Latitude]
   .Points.X        :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XI         :Path   [Longitude,Latitude]
   .Points.XI       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XII        :Path   [Longitude,Latitude]
   .Points.XII      :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XIII       :Path   [Longitude,Latitude]
   .Points.XIII     :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XIV        :Path   [Longitude,Latitude]
   .Points.XIV      :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XV         :Path   [Longitude,Latitude]
   .Points.XV       :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XVI        :Path   [Longitude,Latitude]
   .Points.XVI      :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle)
   .Path.XVII       :Path   [Longitude,Latitude]
   .Points.XVII     :Points   [Longitude,Latitude]   (triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle,triangle_angle

In [15]:
traj_col.trajectories[0].df

,geometry
DateTimeOfPosition,
2023-03-28 06:32:06.247,POINT (5.45945 52.17847)
2023-03-28 10:22:18.600,POINT (5.46825 52.17816)


In [16]:
traj_col.trajectories[1].df

,geometry
DateTimeOfPosition,
2023-03-28 10:44:30.883,POINT (5.46827 52.17810)
2023-03-28 10:46:30.470,POINT (5.46644 52.17542)
2023-03-28 10:48:30.197,POINT (5.45512 52.17663)
2023-03-28 10:48:56.750,POINT (5.45512 52.17663)


In [483]:
traj_col.trajectories[2].df

,geometry,Address
DateTimeOfPosition,,
2023-03-28 11:12:33.617,POINT (5.45511 52.17664),"Kantemarsweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AM, Nederland"
2023-03-28 11:14:24.630,POINT (5.45951 52.17849),"44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland"


In [451]:
dfcpy = df.copy()

## Finalizing the data

In [522]:
data = {}
starts = []
ends = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(traj_col.trajectories):
    starts.append(trajectory.df["Address"].iloc[0])
    ends.append(trajectory.df["Address"].iloc[-1]) 
    
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    
    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
data["start_trip_addr"] = starts
data["end_trip_addr"] = ends

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



## Finalizingggg!

In [523]:
new_df = pd.DataFrame(data)

In [524]:
new_df

,start_trip_addr,end_trip_addr,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,"44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland","Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland",2023-03-28 06:32:06.247,2023-03-28 10:22:18.600,"(5.459454, 52.178471)","(5.468254, 52.178164)",0.979107,03:50:12
1,"Bedrijventerrein Hogenbrink, Hogebrinkerweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AG, Nederland","Kantemarsweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AM, Nederland",2023-03-28 10:44:30.883,2023-03-28 10:48:56.750,"(5.468269, 52.178102)","(5.455119, 52.176635)",1.471204,00:04:25
2,"Kantemarsweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 AM, Nederland","44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland",2023-03-28 11:12:33.617,2023-03-28 11:14:24.630,"(5.455112, 52.176642)","(5.459509, 52.178495)",0.530205,00:01:51
3,"40, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland","1, Prikkebeenhof, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 EX, Nederland",2023-03-28 15:06:24.687,2023-03-28 15:08:25.110,"(5.45955, 52.178429)","(5.460762, 52.175612)",0.339693,00:02:00
4,"1, Prikkebeenhof, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 EX, Nederland","30, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland",2023-03-28 15:12:25.553,2023-03-28 15:14:25.437,"(5.460729, 52.17566)","(5.459459, 52.178149)",0.309593,00:01:59
...,...,...,...,...,...,...,...,...
97,"Oleander, Maimonideslaan, De Orangerie, Amstelveen, Noord-Holland, Nederland, 1186 DV, Nederland","44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland",2023-04-28 12:57:02.930,2023-04-28 13:36:52.403,"(4.869734, 52.295627)","(5.459535, 52.178487)",66.853649,00:39:49
98,"40, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland","Subway, 2, Wiekenweg, Valleipoort, Amersfoort, Utrecht, Nederland, 3815 KL, Nederland",2023-04-28 15:20:04.613,2023-04-28 15:26:04.340,"(5.459449, 52.178427)","(5.423039, 52.163166)",4.386913,00:05:59
99,"Barneveldseweg, Hogeweg, Valleipoort, Amersfoort, Utrecht, Nederland, 3815 KP, Nederland","44, Groot Middendorp, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PW, Nederland",2023-04-28 15:38:04.677,2023-04-28 15:45:40.523,"(5.425032, 52.162924)","(5.459486, 52.1785)",4.201214,00:07:35
100,"35, Weidelaan, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 PP, Nederland","13, Veenslagenweg, Hoevelaken, Nijkerk, Gelderland, Nederland, 3871 NA, Nederland",2023-04-30 18:59:46.677,2023-04-30 22:12:27.780,"(5.460566, 52.178856)","(5.457597, 52.177229)",0.376065,03:12:41


In [518]:
durations[0]

Components(days=0, hours=3, minutes=50, seconds=12, milliseconds=353, microseconds=0, nanoseconds=0)

In [513]:
len(starts), len(ends)

(102, 102)

In [390]:
counts = 0
for col in traj_col.trajectories:
    counts += len(col.df)

In [391]:
counts

823

In [392]:
986 - 101

885

## Analysis

In [287]:
trjj = set(traj.df.index.values)

In [288]:
datet = set(df.DateTimeOfPosition.values)

In [290]:
len(datet)

986

In [302]:
uniqueList = []
duplicateList = []
 
for idx, i in enumerate(df.DateTimeOfPosition.values):
    if i not in uniqueList:
        uniqueList.append(i)
    elif i not in duplicateList:
        print("dupliacte found : ", idx, i)
        duplicateList.append(i)

dupliacte found :  175 2023-04-24T09:19:52.923000000
dupliacte found :  420 2023-04-13T13:13:38.450000000
dupliacte found :  754 2023-04-04T12:38:02.243000000
dupliacte found :  957 2023-03-29T12:55:57.223000000
dupliacte found :  997 2023-03-28T08:22:18.600000000


In [292]:
uniqueList

[numpy.datetime64('2023-05-01T04:29:18.987000000'),
 numpy.datetime64('2023-05-01T04:28:58.437000000'),
 numpy.datetime64('2023-05-01T04:26:58.420000000'),
 numpy.datetime64('2023-05-01T04:24:58.953000000'),
 numpy.datetime64('2023-05-01T04:22:58.207000000'),
 numpy.datetime64('2023-05-01T04:20:58.480000000'),
 numpy.datetime64('2023-05-01T04:16:57.950000000'),
 numpy.datetime64('2023-05-01T04:14:58.670000000'),
 numpy.datetime64('2023-05-01T04:13:11.100000000'),
 numpy.datetime64('2023-05-01T04:11:32.010000000'),
 numpy.datetime64('2023-05-01T03:14:26.343000000'),
 numpy.datetime64('2023-05-01T00:54:30.643000000'),
 numpy.datetime64('2023-04-30T20:12:27.780000000'),
 numpy.datetime64('2023-04-30T16:59:46.677000000'),
 numpy.datetime64('2023-04-30T08:45:35.667000000'),
 numpy.datetime64('2023-04-30T07:38:31.283000000'),
 numpy.datetime64('2023-04-30T03:57:38.340000000'),
 numpy.datetime64('2023-04-29T15:57:34.850000000'),
 numpy.datetime64('2023-04-29T05:55:56.260000000'),
 numpy.datet

In [293]:
duplicateList

[numpy.datetime64('2023-04-24T09:19:52.923000000'),
 numpy.datetime64('2023-04-13T13:13:38.450000000'),
 numpy.datetime64('2023-04-04T12:38:02.243000000'),
 numpy.datetime64('2023-03-29T12:55:57.223000000'),
 numpy.datetime64('2023-03-28T08:22:18.600000000')]

In [300]:
df[920:970]

,UnitPositionId,DateTimeOfPosition,IgnitionOn,long,lat
920,631259941,2023-03-31 08:38:56.167000+02:00,1,52.119372,5.497375
921,631259356,2023-03-31 08:36:56.363000+02:00,1,52.119449,5.507664
922,631258794,2023-03-31 08:34:57.190000+02:00,1,52.120682,5.504580
923,631239620,2023-03-31 07:11:06.760000+02:00,0,52.120832,5.504574
924,631239616,2023-03-31 07:11:05.590000+02:00,0,52.120832,5.504574
925,631239110,2023-03-31 07:09:04.960000+02:00,1,52.120564,5.504465
926,631237953,2023-03-31 07:04:24.963000+02:00,0,52.120555,5.504439
927,631237530,2023-03-31 07:02:44.897000+02:00,1,52.119690,5.506430
928,631237025,2023-03-31 07:00:45.270000+02:00,1,52.118877,5.524479
929,631236464,2023-03-31 06:58:45.310000+02:00,1,52.121646,5.547970


## Experiments

In [308]:
!pip install haversine


In [309]:
from haversine import haversine, Unit


In [327]:
locA = (5.463401, 52.176882 )
locB = ( 5.468272, 52.178172)

In [328]:
haversine(locA, locB, unit=Unit.METERS)

560.1367889147838

In [307]:
df[970:]

,UnitPositionId,DateTimeOfPosition,IgnitionOn,long,lat
970,630805534,2023-03-29 10:06:00.637000+02:00,1,52.224869,5.485091
971,630782507,2023-03-29 09:06:01.667000+02:00,0,52.224870,5.485137
972,630781879,2023-03-29 09:04:23.663000+02:00,1,52.225156,5.485039
973,630781171,2023-03-29 09:02:24.477000+02:00,1,52.222697,5.482524
974,630780455,2023-03-29 09:00:24.053000+02:00,1,52.217752,5.478537
975,630779715,2023-03-29 08:58:24.337000+02:00,1,52.207939,5.467498
976,630778991,2023-03-29 08:56:24.553000+02:00,1,52.197212,5.453079
977,630778283,2023-03-29 08:54:23.940000+02:00,1,52.187074,5.449514
978,630777570,2023-03-29 08:52:24.377000+02:00,1,52.179982,5.454872
979,630776865,2023-03-29 08:50:23.957000+02:00,1,52.178341,5.459455


In [317]:
stops = detector.get_stop_points(min_duration=timedelta(seconds=600), max_diameter=20)


In [318]:
stops


,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-03-28 10:22:18.600000,POINT (5.46826 52.17816),2023-03-28 10:22:18.600,2023-03-28 10:44:30.883,Allsetra,1332.283
Allsetra_2023-03-28 10:48:56.750000,POINT (5.45512 52.17664),2023-03-28 10:48:56.750,2023-03-28 11:12:33.617,Allsetra,1416.867
Allsetra_2023-03-28 11:14:24.630000,POINT (5.45953 52.17846),2023-03-28 11:14:24.630,2023-03-28 15:06:24.687,Allsetra,13920.057
Allsetra_2023-03-28 15:15:09.780000,POINT (5.45956 52.17841),2023-03-28 15:15:09.780,2023-03-28 19:46:49.237,Allsetra,16299.457
Allsetra_2023-03-29 09:06:01.667000,POINT (5.48511 52.22487),2023-03-29 09:06:01.667,2023-03-29 10:06:00.637,Allsetra,3598.970
...,...,...,...,...,...
Allsetra_2023-04-28 08:30:42.683000,POINT (5.25027 51.34076),2023-04-28 08:30:42.683,2023-04-28 09:38:38.617,Allsetra,4075.934
Allsetra_2023-04-28 11:26:40.040000,POINT (5.45946 52.17845),2023-04-28 11:26:40.040,2023-04-28 11:48:16.607,Allsetra,1296.567
Allsetra_2023-04-28 13:36:52.403000,POINT (5.45949 52.17846),2023-04-28 13:36:52.403,2023-04-28 15:20:04.613,Allsetra,6192.210


In [362]:
dfcpy = df.copy()

In [363]:
dfcpy.sort_values("DateTimeOfPosition", ascending=True)

,UnitPositionId,DateTimeOfPosition,IgnitionOn,long,lat
999,630477643,2023-03-28 06:32:06.247000+02:00,0,52.178471,5.459454
996,630553514,2023-03-28 10:22:18.600000+02:00,1,52.178164,5.468254
998,630553512,2023-03-28 10:22:18.600000+02:00,1,52.178559,5.459197
997,630553513,2023-03-28 10:22:18.600000+02:00,1,52.176882,5.463401
995,630553518,2023-03-28 10:22:19.030000+02:00,1,52.178164,5.468254
...,...,...,...,...,...
4,637456653,2023-05-01 06:22:58.207000+02:00,1,52.157829,5.404613
3,637457374,2023-05-01 06:24:58.953000+02:00,1,52.152371,5.388205
2,637458106,2023-05-01 06:26:58.420000+02:00,1,52.158796,5.379862
1,637458804,2023-05-01 06:28:58.437000+02:00,1,52.154865,5.376504


In [364]:
dfcpy['HaversineDistance'] = None

# Iterate through the DataFrame rows
for i in range(1, len(dfcpy)):
    # Get the coordinates of the current and previous points
    lon1, lat1 = dfcpy.loc[i, 'long'], dfcpy.loc[i, 'lat']
    lon2, lat2 = dfcpy.loc[i-1, 'long'], dfcpy.loc[i-1, 'lat']
    
    # Calculate the Haversine distance between the points
    dist = haversine((lat1, lon1), (lat2, lon2), unit=Unit.METERS)
    
    # Update the Haversine distance in the DataFrame
    dfcpy.at[i, 'HaversineDistance'] = dist


In [411]:
dfcpy[950:]


,UnitPositionId,DateTimeOfPosition,IgnitionOn,long,lat,HaversineDistance
950,631147104,2023-03-30 15:27:15.267000+02:00,0,52.178647,5.459322,100.539065
951,631132434,2023-03-30 14:13:52.800000+02:00,0,52.178837,5.459460,26.034194
952,630953760,2023-03-30 03:52:45.730000+02:00,0,52.178414,5.459761,57.554628
953,630905529,2023-03-29 15:52:42.437000+02:00,0,52.178459,5.459490,30.542774
954,630900726,2023-03-29 15:19:59.210000+02:00,0,52.178489,5.459529,5.461991
955,630900493,2023-03-29 15:18:23.500000+02:00,1,52.176455,5.456319,422.011742
956,630896417,2023-03-29 14:55:57.223000+02:00,0,52.176504,5.456319,5.423871
957,630896416,2023-03-29 14:55:57.223000+02:00,0,52.176504,5.456319,0.0
958,630895964,2023-03-29 14:53:52.047000+02:00,1,52.178589,5.459354,408.84607
959,630865927,2023-03-29 13:07:14.520000+02:00,0,52.178594,5.459523,18.800117


In [22]:
mp = leafmap.Map()

In [23]:
mp.add_gdf(traj_col.trajectories[0].to_point_gdf() , fill_colors=["blue", "red"])

In [26]:
mp

Map(center=[52.178317500000006, 5.4638539999999995], controls=(ZoomControl(options=['position', 'zoom_in_text'…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

## Findings

### 1) Discrepancy b/w some timestamps and distance covered. 
### 2) x != long and y != lat. their values are flipped. otherwise, displays wrong location. 
### 3) Duplicates